In [3]:
# Author: Sara_Fraija
import os
import pandas as pd

# Configuración de rutas
GRBsINFO = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
PATH_GRBs_Healpix = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpix_maps/'
PATH_GENERAL = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/'
PATH_CODES = os.path.join(PATH_GENERAL, 'scripts')
LABELS = ['1', '2']
bin_size=0.25##PSF
window_size=bin_size*2
# Leer el archivo CSV
GRBs = pd.read_csv(GRBsINFO)

def generate_bash_script(grb, ra, dec, bin_size, path):
    """Genera un script Bash para un GRB específico."""
    PATH_GRBs = os.path.join(path, grb)
    hist_path = os.path.join(PATH_GENERAL, 'SIG_ALL_GRBs')
    os.makedirs(hist_path, exist_ok=True)
    bin_size = max(bin_size, GRBs.loc[GRBs['Name'] == grb, 'Error_Radius'].values[0])

    script = f"""#!/usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_GRBs}/{grb}/maps/codes/HIST.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL{grb}
# Author: Sara_Fraija

source /data/disk01/home/smfraijac/.bashrc
hawc_aerie

RA={ra}
DEC={dec}
Bin_size={bin_size}
Hist_path={hist_path}
Window_size={window_size}
for Label in {LABELS[0]} {LABELS[1]}; do
    GRB={grb}_$Label
    output_file={hist_path}/{grb}_PSF_{bin_size}_Hist$Label.txt
    max_output_file={PATH_GENERAL}/MaxSigHist_{bin_size}_$Label.txt
    rm -f "$output_file"

    transit_file={PATH_GRBs}/transit_$Label/{grb}_transit_$Label.fits.gz
    # PL=$(makeSignificanceHistogram.py "$transit_file" --ra "$RA" --dec "$DEC" --includeArea --batch --binsize "$Bin_size" -o "$Hist_path/$GRB.png")
    PL=$(plotMercator.py "$transit_file"  --magma --cross  --circle $RA $DEC $Bin_size --circleproperty white 2 1 --origin $RA $DEC $Window_size $Window_size -o "$Hist_path/$GRB.png")
    echo -e "$PL" > "$output_file"

    Maxi=$(grep "Max" "$output_file")
    echo -e "{grb},$Label,$Maxi" >> "$max_output_file"
done
"""
    return script

def create_script_file(content, grb, bin_size):
    """Crea y guarda el script Bash en un archivo."""
    file_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        file.write(content)

def generate_scripts_for_all_grbs(bin_size, path):
    """Genera scripts para todos los GRBs en el archivo CSV."""
    for _, row in GRBs.iterrows():
        grb, ra, dec = row['Name'], row['Ra'], row['Dec']
        script_content = generate_bash_script(grb, ra, dec, bin_size, path)
        create_script_file(script_content, grb, bin_size)

def generate_batch_sender(bin_size):
    """Genera un script para ejecutar todos los scripts generados."""
    batch_file_path = os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')
    os.makedirs(os.path.dirname(batch_file_path), exist_ok=True)
    with open(batch_file_path, 'w') as batch_file:
        batch_file.write(f'rm -f {PATH_GENERAL}/MaxSig_{bin_size}_*.txt\n')
        for grb in GRBs['Name']:
            script_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
            batch_file.write(f'sh {script_path}\n')

# Generar scripts para bin_size = 0.6

generate_scripts_for_all_grbs(bin_size, PATH_GRBs_Healpix)
generate_batch_sender(bin_size)
import subprocess
subprocess.run(["bash",os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')])
import pandas as pd
import numpy as np
path='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
coordi=pd.read_csv(path);
# Lista para almacenar los DataFrames
transit=1
# Ruta del archivo y procesamiento
for transit in [1,2]:
    dfs = []    
    for psf in [bin_size,0.70,0.8]:
        path = f'../MaxSigHist_{psf}_{transit}.txt'
        
        # Leer el archivo de texto
        with open(path, 'r') as file:
            lines = file.readlines()
        
        # Procesar las líneas para extraer los datos
        data = []
        for line in lines:
            # Eliminar espacios en blanco al inicio y final de la línea
            line = line = ' '.join(line.split())
            line = line.replace('Max:', '').replace(')', '').replace('(', '').replace(' ', ',').replace(',,', ',')
            # Reemplazar y dividir la línea
            
            line = line.split(',')
            # print(line)
            # Agregar a la lista de datos
            data.append(line)
        
        # Convertir a un array de numpy y transponer
        data = np.array(data).T
        
        # Crear un DataFrame
        df = pd.DataFrame()
        columns = ['Name','transit', 'Significance','RA', 'DEC',]
        
        for i, Data in zip(columns, data):
            df[i] = Data
        
        #Agregar una columna para identificar el valor de psf
        df['PSF'] = psf
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
#!/usr/bin/env python3
# AUTHOR: Sara Fraija
# This script allows generating bash files to compute upper limits for each GRB. 
# You can change the GRB list, energy ranges, and EBL model.

import pandas as pd
import subprocess
import os
# ------------------------------------------------------------------------
# PATHS & FILENAMES
# ------------------------------------------------------------------------
energy_ranges = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Energy_ranges.csv'
GRBINFO_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/'
OUTPUT_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/files/2.5/PSF_0.25/'
os.makedirs(OUTPUT_folder,exist_ok=True)
PATH_SH = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/'
RECALCULATED = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/corrected/'

# Leer Energy_ranges.csv con nombres de columnas correctos
Energy_ranges = pd.read_csv(energy_ranges, names=['Name', 'Emin', 'Emax'])
Energy_ranges = Energy_ranges.set_index('Name')
Energy_ranges.sort_values(by='Name')
# ------------------------------------------------------------------------
# Function to generate bash script
# ------------------------------------------------------------------------
def script(GRBINFO, transit, PATH_SH, OUTPUT_folder, RECALCULATED, EBL, PSF):
    OUTPUT_FILE = f"{OUTPUT_folder}UpperLimit_{transit}_{EBL}.csv"

    # Leer archivo CSV con GRB information
    Data = pd.read_csv(GRBINFO)
    Data.sort_values('Name')
    # Verificar si la columna "Name" existe en el CSV
    if "Name" not in Data.columns:
        print(f"Error: La columna 'Name' no está en {GRBINFO}. Verifica el formato del archivo.")
        print(f"Columnas disponibles: {Data.columns}")
        return

    # Usar "Name" como índice
    Data = Data.set_index("Name")

    # Asignar valores de Emin y Emax si existen en Energy_ranges
    Data['Emin'] = Data.index.map(Energy_ranges['Emin'])
    Data['Emax'] = Data.index.map(Energy_ranges['Emax'])
    # Verificar si hay valores NaN en Emin o Emax
    if Data[['Emin', 'Emax']].isna().any().any():
        print("Error: Algunos GRBs no tienen valores de Emin o Emax. Verifica Energy_ranges.csv.")
        print(Data[['Emin', 'Emax']].isna().sum())
        return


    line = ""
    for grb in Data.index:  
        row = Data.loc[grb]
        RA = row['RA']
        Dec = row['DEC']
        z = row['z']
        
        index = 2.07 if grb == 'GRB170817529' else 2.5

        Emin = row['Emin']
        Emax = row['Emax']
        line += f'PL="PowerLaw,1.0e-10,{index}"\n'
        line_1=f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB0C1_N1024.fits.gz $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{1..10}}C{{0..1}}_N1024.fits.gz -b B0C1 B{{1..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line_2=f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{0..10}}C{{0..1}}_N1024.fits.gz -b B{{0..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line+=line_1 if grb=='GRB150110923'else line_2
        line += f'echo -e "{grb},${{ULs}}" >> $output_file\n'
        

    # Código base del script Bash
    code = f"""#! /usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_SH}/logs/UL.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL
# Author: Sara Fraija
source /data/disk01/home/smfraijac/.bashrc
hawc_aerie
output_file={OUTPUT_FILE}
DR51=/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/config/pass5.f_DR/zebra-100pct-FHit.root
dir={RECALCULATED}\n
output_file={OUTPUT_FILE}\n
rm -f output_file
"""
    # Guardar el script Bash
    file_path = f"{PATH_SH}UL{transit}_{EBL}_{PSF}.sh"
    with open(file_path, 'w') as bash_file:
        bash_file.write(code + line)
    print(f"Script generado: {file_path}")
# ------------------------------------------------------------------------
# Running script
# ------------------------------------------------------------------------
EBL_list = ['Franceschini08', 'Gilmore12Fiducial']

# for EBL in EBL_list:
#     File_1st_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates_with_Max_Sig_1_00.25.csv'
#     File_2nd_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates_with_Max_Sig_2_00.25.csv'
#     script(File_1st_transit, transit=1, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
#     script(File_2nd_transit, transit=2, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
#     p1 = subprocess.Popen(['bash', f"{PATH_SH}UL1_{EBL}_0.25.sh"])
#     p2 = subprocess.Popen(['bash', f"{PATH_SH}UL2_{EBL}_0.25.sh"])

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


...

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line
/tmp/ipykernel_2362739/208895439.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data).T
/tmp/ipykernel_2362739/208895439.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data).T
/tmp/ipykernel_2362739/208895439.py:116: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must speci

ValueError: Length of values (3) does not match length of index (5)

In [ ]:
# import pandas as pd

# # Ruta del archivo
# path = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/MaxSigHist_0.6_1.txt'

# # Leer el archivo de texto
# with open(path, 'r') as file:
#     lines = file.readlines()

# # Procesar las líneas para extraer los datos
# data = []
# for line in lines:
#     # Eliminar espacios en blanco al inicio y final de la línea
#     line = line.strip()
#     print(line)
    
#     # Verificar si la línea tiene el formato esperado
#     if not line:
#         continue  # Saltar líneas vacías
    
#     try:
#         # Dividir la línea en partes usando la coma como separador
#         parts = line.split(',')
#         if len(parts) < 3:
#             print(f"Línea ignorada (formato incorrecto): {line}")
#             continue
        
#         grb_id = parts[0]
        
#         # Extraer las coordenadas y la significancia
#         coords_part = parts[2].split('(')
#         if len(coords_part) < 2:
#             print(f"Línea ignorada (formato de coordenadas incorrecto): {line}")
#             continue
        
#         coords_part = coords_part[1].split(')')[0]  # Extraer "(RA, Dec)"
#         coords = coords_part.split(',')  # Dividir en RA y Dec
        
#         if len(coords) < 2:
#             print(f"Línea ignorada (coordenadas incompletas): {line}")
#             continue
        
#         # Eliminar espacios en blanco y convertir a float
#         ra = float(coords[0].strip())
#         dec = float(coords[1].strip())
        
#         # Extraer la significancia
#         significance_part = parts[2].split('is ')
#         if len(significance_part) < 2:
#             print(f"Línea ignorada (formato de significancia incorrecto): {line}")
#             continue
        
#         significance = float(significance_part[1].strip())  # Convertir a float
        
#         # Agregar los datos a la lista
#         data.append([grb_id, ra, dec, significance])
    
#     except Exception as e:
#         print(f"Error procesando la línea: {line}")
#         print(f"Detalles del error: {e}")
#         continue

# # Crear un DataFrame con los datos
# df = pd.DataFrame(data, columns=['GRB ID', 'RA_max', 'Dec_max', 'Significance'])

# # Mostrar la tabla
# print(df)


In [28]:
import pandas as pd
import numpy as np
path='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
coordi=pd.read_csv(path);
# Lista para almacenar los DataFrames
transit=1
# Ruta del archivo y procesamiento
for transit in [1,2]:
    dfs = []    
    for psf in [bin_size,0.7,0.8]:
        path = f'../MaxSigHist_{psf}_{transit}.txt'
        
        # Leer el archivo de texto
        with open(path, 'r') as file:
            lines = file.readlines()
        
        # Procesar las líneas para extraer los datos
        data = []
        for line in lines:
            # Eliminar espacios en blanco al inicio y final de la línea
            line = line = ' '.join(line.split())
            line = line.replace('Max:', '').replace(')', '').replace('(', '').replace(' ', ',').replace(',,', ',')
            # Reemplazar y dividir la línea
            
            line = line.split(',')
            # print(line)
            # Agregar a la lista de datos
            data.append(line)
        
        # Convertir a un array de numpy y transponer
        data = np.array(data).T
        
        # Crear un DataFrame
        df = pd.DataFrame()
        columns = ['Name','transit', 'Significance','RA', 'DEC',]
        
        for i, Data in zip(columns, data):
            df[i] = Data
        
        #Agregar una columna para identificar el valor de psf
        df['PSF'] = psf
        
        # Agregar el DataFrame a la lista
        dfs.append(df)

    # Combinar todos los DataFrames en uno solo
    combined_df = pd.concat(dfs, ignore_index=True).drop_duplicates()
    C=combined_df.join(coordi.set_index('Name'),on='Name')
    C.to_csv(f'../config/Coordinates_with_Max_Sig_{transit}_0{bin_size}.csv')
    # Mostrar el DataFrame combinado
    print(C)

             Name transit Significance      RA     DEC   PSF       Ra  \
0    GRB150110923       1         1.10  289.64   32.71  0.25  289.360   
1    GRB150922234       1         2.15  294.43   -5.27  0.25  294.381   
2    GRB160726065       1         1.24   98.96   -6.92  0.25   98.821   
3    GRB170325331       1         1.03  127.27   20.78  0.25  127.483   
4    GRB180715755       1         0.93  235.28   -1.16  0.25  235.069   
5    GRB180718082       1         1.29  336.31    2.69  0.25  336.019   
6    GRB181125371       1         0.74  268.55   -2.54  0.25  268.413   
7    GRB190427190       1         1.85  279.98   40.13  0.25  280.213   
8    GRB201008443       1         1.70  162.10   45.83  0.25  161.859   
9    GRB201221963       1         0.00  170.80   42.41  0.25  171.059   
10   GRB210323918       1         1.56  317.72   25.65  0.25  317.946   
11   GRB170817529       1         2.53  197.67  -23.44  0.25  197.500   
66   GRB160726065       1         1.21   98.96   -6

/tmp/ipykernel_2168043/3541833435.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data).T


In [34]:
# Author: Sara_Fraija
import os
import pandas as pd

# Configuración de rutas
GRBsINFO = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
PATH_GRBs_Healpix = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpix_maps/'
PATH_GENERAL = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/'
PATH_CODES = os.path.join(PATH_GENERAL, 'scripts')
LABELS = ['1', '2']
bin_size=0.25##PSF
window_size=bin_size*2
# Leer el archivo CSV
GRBs = pd.read_csv(GRBsINFO)

def generate_bash_script(grb, ra, dec, bin_size, path):
    """Genera un script Bash para un GRB específico."""
    PATH_GRBs = os.path.join(path, grb)
    hist_path = os.path.join(PATH_GENERAL, 'SIG_ALL_GRBs')
    os.makedirs(hist_path, exist_ok=True)
    bin_size = max(bin_size, GRBs.loc[GRBs['Name'] == grb, 'Error_Radius'].values[0])

    script = f"""#!/usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_GRBs}/{grb}/maps/codes/HIST.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL{grb}
# Author: Sara_Fraija

source /data/disk01/home/smfraijac/.bashrc
hawc_aerie

RA={ra}
DEC={dec}
Bin_size={bin_size}
Hist_path={hist_path}
Window_size={window_size}
for Label in {LABELS[0]} {LABELS[1]}; do
    GRB={grb}_$Label
    output_file={hist_path}/{grb}_PSF_{bin_size}_Hist$Label.txt
    max_output_file={PATH_GENERAL}/MaxSigHist_{bin_size}_$Label.txt
    rm -f "$output_file"

    transit_file={PATH_GRBs}/transit_$Label/{grb}_transit_$Label.fits.gz
    # PL=$(makeSignificanceHistogram.py "$transit_file" --ra "$RA" --dec "$DEC" --includeArea --batch --binsize "$Bin_size" -o "$Hist_path/$GRB.png")
    PL=$(plotMercator.py "$transit_file"  --magma --cross  --circle $RA $DEC $Bin_size --circleproperty white 2 1 --origin $RA $DEC $Window_size $Window_size -o "$Hist_path/$GRB.png")
    echo -e "$PL" > "$output_file"

    Maxi=$(grep "Max" "$output_file")
    echo -e "{grb},$Label,$Maxi" >> "$max_output_file"
done
"""
    return script

def create_script_file(content, grb, bin_size):
    """Crea y guarda el script Bash en un archivo."""
    file_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        file.write(content)

def generate_scripts_for_all_grbs(bin_size, path):
    """Genera scripts para todos los GRBs en el archivo CSV."""
    for _, row in GRBs.iterrows():
        grb, ra, dec = row['Name'], row['Ra'], row['Dec']
        script_content = generate_bash_script(grb, ra, dec, bin_size, path)
        create_script_file(script_content, grb, bin_size)

def generate_batch_sender(bin_size):
    """Genera un script para ejecutar todos los scripts generados."""
    batch_file_path = os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')
    os.makedirs(os.path.dirname(batch_file_path), exist_ok=True)
    with open(batch_file_path, 'w') as batch_file:
        batch_file.write(f'rm -f {PATH_GENERAL}/MaxSig_{bin_size}_*.txt\n')
        for grb in GRBs['Name']:
            script_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
            batch_file.write(f'sh {script_path}\n')

# Generar scripts para bin_size = 0.6

generate_scripts_for_all_grbs(bin_size, PATH_GRBs_Healpix)
generate_batch_sender(bin_size)
import subprocess
subprocess.run(["bash",os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')])

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


...

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


CompletedProcess(args=['bash', '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/scripts/BatchSenderMax_0.25.sh'], returncode=0)

In [40]:
import pandas as pd
import numpy as np
path='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
coordi=pd.read_csv(path);
# Lista para almacenar los DataFrames
transit=1
# Ruta del archivo y procesamiento
for transit in [1,2]:
    dfs = []    
    for psf in [bin_size,0.70,0.8]:
        path = f'../MaxSigHist_{psf}_{transit}.txt'
        
        # Leer el archivo de texto
        with open(path, 'r') as file:
            lines = file.readlines()
        
        # Procesar las líneas para extraer los datos
        data = []
        for line in lines:
            # Eliminar espacios en blanco al inicio y final de la línea
            line = line = ' '.join(line.split())
            line = line.replace('Max:', '').replace(')', '').replace('(', '').replace(' ', ',').replace(',,', ',')
            # Reemplazar y dividir la línea
            
            line = line.split(',')
            # print(line)
            # Agregar a la lista de datos
            data.append(line)
        
        # Convertir a un array de numpy y transponer
        data = np.array(data).T
        
        # Crear un DataFrame
        df = pd.DataFrame()
        columns = ['Name','transit', 'Significance','RA', 'DEC',]
        
        for i, Data in zip(columns, data):
            df[i] = Data
        
        #Agregar una columna para identificar el valor de psf
        df['PSF'] = psf
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
#!/usr/bin/env python3
# AUTHOR: Sara Fraija
# This script allows generating bash files to compute upper limits for each GRB. 
# You can change the GRB list, energy ranges, and EBL model.

import pandas as pd
import subprocess
import os
# ------------------------------------------------------------------------
# PATHS & FILENAMES
# ------------------------------------------------------------------------
energy_ranges = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Energy_ranges.csv'
GRBINFO_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/'
OUTPUT_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/files/2.5/PSF_0.25/'
os.makedirs(OUTPUT_folder,exist_ok=True)
PATH_SH = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/'
RECALCULATED = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/corrected/'

# Leer Energy_ranges.csv con nombres de columnas correctos
Energy_ranges = pd.read_csv(energy_ranges, names=['Name', 'Emin', 'Emax'])
Energy_ranges = Energy_ranges.set_index('Name')
Energy_ranges.sort_values(by='Name')
# ------------------------------------------------------------------------
# Function to generate bash script
# ------------------------------------------------------------------------
def script(GRBINFO, transit, PATH_SH, OUTPUT_folder, RECALCULATED, EBL, PSF):
    OUTPUT_FILE = f"{OUTPUT_folder}UpperLimit_{transit}_{EBL}.csv"

    # Leer archivo CSV con GRB information
    Data = pd.read_csv(GRBINFO)
    Data.sort_values('Name')
    # Verificar si la columna "Name" existe en el CSV
    if "Name" not in Data.columns:
        print(f"Error: La columna 'Name' no está en {GRBINFO}. Verifica el formato del archivo.")
        print(f"Columnas disponibles: {Data.columns}")
        return

    # Usar "Name" como índice
    Data = Data.set_index("Name")

    # Asignar valores de Emin y Emax si existen en Energy_ranges
    Data['Emin'] = Data.index.map(Energy_ranges['Emin'])
    Data['Emax'] = Data.index.map(Energy_ranges['Emax'])
    # Verificar si hay valores NaN en Emin o Emax
    if Data[['Emin', 'Emax']].isna().any().any():
        print("Error: Algunos GRBs no tienen valores de Emin o Emax. Verifica Energy_ranges.csv.")
        print(Data[['Emin', 'Emax']].isna().sum())
        return


    line = ""
    for grb in Data.index:  
        row = Data.loc[grb]
        RA = row['RA']
        Dec = row['DEC']
        z = row['z']
        
        index = 2.07 if grb == 'GRB170817529' else 2.5

        Emin = row['Emin']
        Emax = row['Emax']
        line += f'PL="PowerLaw,1.0e-10,{index}"\n'
        line_1=f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB0C1_N1024.fits.gz $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{1..10}}C{{0..1}}_N1024.fits.gz -b B0C1 B{{1..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line_2=f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{0..10}}C{{0..1}}_N1024.fits.gz -b B{{0..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line+=line_1 if grb=='GRB150110923'else line_2
        line += f'echo -e "{grb},${{ULs}}" >> $output_file\n'
        

    # Código base del script Bash
    code = f"""#! /usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_SH}/logs/UL.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL
# Author: Sara Fraija
source /data/disk01/home/smfraijac/.bashrc
hawc_aerie
output_file={OUTPUT_FILE}
DR51=/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/config/pass5.f_DR/zebra-100pct-FHit.root
dir={RECALCULATED}\n
output_file={OUTPUT_FILE}\n
"""
    # Guardar el script Bash
    file_path = f"{PATH_SH}UL{transit}_{EBL}_{PSF}.sh"
    with open(file_path, 'w') as bash_file:
        bash_file.write(code + line)
    print(f"Script generado: {file_path}")
# ------------------------------------------------------------------------
# Running script
# ------------------------------------------------------------------------
EBL_list = ['Franceschini08', 'Gilmore12Fiducial']

for EBL in EBL_list:
    File_1st_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/MaxSigHist_0.25_1.txt'
    File_2nd_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/MaxSigHist_0.25_2.txt'
    script(File_1st_transit, transit=1, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
    script(File_2nd_transit, transit=2, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
    p1 = subprocess.Popen(['bash', f"{PATH_SH}UL1_{EBL}_0.25.sh"])
    p2 = subprocess.Popen(['bash', f"{PATH_SH}UL2_{EBL}_0.25.sh"])

KeyError: 'Name'

In [8]:
# Author: Sara_Fraija
import os
import pandas as pd

# Configuración de rutas
GRBsINFO = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
PATH_GRBs_Healpix = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpix_maps/'
PATH_GENERAL = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/'
PATH_CODES = os.path.join(PATH_GENERAL, 'scripts')
LABELS = ['1', '2']
bin_size = 0.25  ##PSF
window_size = bin_size * 2
# Leer el archivo CSV
GRBs = pd.read_csv(GRBsINFO)
for i in []
def generate_bash_script(grb, ra, dec, bin_size, path):
    """Genera un script Bash para un GRB específico."""
    PATH_GRBs = os.path.join(path, grb)
    hist_path = os.path.join(PATH_GENERAL, 'SIG_ALL_GRBs')
    os.makedirs(hist_path, exist_ok=True)
    bin_size = max(bin_size, GRBs.loc[GRBs['Name'] == grb, 'Error_Radius'].values[0])

    script = f"""#!/usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_GRBs}/{grb}/maps/codes/HIST.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL{grb}
# Author: Sara_Fraija

source /data/disk01/home/smfraijac/.bashrc
hawc_aerie

RA={ra}
DEC={dec}
Bin_size={bin_size}
Hist_path={hist_path}
Window_size={window_size}
for Label in {LABELS[0]} {LABELS[1]}; do
    GRB={grb}_$Label
    output_file={hist_path}/{grb}_PSF_{bin_size}_Hist$Label.txt
    max_output_file={PATH_GENERAL}/MaxSigHist_{bin_size}_$Label.txt
    rm -f "$output_file"

    transit_file={PATH_GRBs}/transit_$Label/{grb}_transit_$Label.fits.gz
    # PL=$(makeSignificanceHistogram.py "$transit_file" --ra "$RA" --dec "$DEC" --includeArea --batch --binsize "$Bin_size" -o "$Hist_path/$GRB.png")
    PL=$(plotMercator.py "$transit_file"  --magma --cross  --circle $RA $DEC $Bin_size --circleproperty white 2 1 --origin $RA $DEC $Window_size $Window_size -o "$Hist_path/$GRB.png")
    echo -e "$PL" > "$output_file"

    Maxi=$(grep "Max" "$output_file")
    echo -e "{grb},$Label,$Maxi" >> "$max_output_file"
done
"""
    return script

def create_script_file(content, grb, bin_size):
    """Crea y guarda el script Bash en un archivo."""
    file_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if os.path.exists(file_path):
        os.remove(file_path)
    with open(file_path, 'w') as file:
        file.write(content)

def generate_scripts_for_all_grbs(bin_size, path):
    """Genera scripts para todos los GRBs en el archivo CSV."""
    for _, row in GRBs.iterrows():
        grb, ra, dec = row['Name'], row['Ra'], row['Dec']
        script_content = generate_bash_script(grb, ra, dec, bin_size, path)
        create_script_file(script_content, grb, bin_size)

def generate_batch_sender(bin_size):
    """Genera un script para ejecutar todos los scripts generados."""
    batch_file_path = os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')
    os.makedirs(os.path.dirname(batch_file_path), exist_ok=True)
    if os.path.exists(batch_file_path):
        os.remove(batch_file_path)
    with open(batch_file_path, 'w') as batch_file:
        batch_file.write(f'rm -f {PATH_GENERAL}/MaxSig_{bin_size}_*.txt\n')
        for grb in GRBs['Name']:
            script_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
            batch_file.write(f'sh {script_path}\n')

# Generar scripts para bin_size = 0.6

generate_scripts_for_all_grbs(bin_size, PATH_GRBs_Healpix)
generate_batch_sender(bin_size)
import subprocess
subprocess.run(["bash", os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')])

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


...

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


CompletedProcess(args=['bash', '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/scripts/BatchSenderMax_0.25.sh'], returncode=0)

In [7]:

import pandas as pd
import numpy as np
path = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/GRBs_list.csv'
coordi = pd.read_csv(path)
# Lista para almacenar los DataFrames
transit = 1
# Ruta del archivo y procesamiento
for transit in [1, 2]:
    dfs = []    
    for psf in [bin_size, 0.70, 0.8]:
        path = f'../MaxSigHist_{psf}_{transit}.txt'
        # Leer el archivo de texto
        with open(path, 'r') as file:
            lines = file.readlines()
        
        # Procesar las líneas para extraer los datos
        data = []
        for line in lines:
            # Eliminar espacios en blanco al inicio y final de la línea
            line = ' '.join(line.split())
            line = line.replace('Max:', '').replace(')', '').replace('(', '').replace(' ', ',').replace(',,', ',')
            # Reemplazar y dividir la línea
            line = line.split(',')
            # Agregar a la lista de datos
            data.append(line)
        
        # Convertir a un array de numpy y transponer
        data = np.array(data).T
        
        # Crear un DataFrame
        df = pd.DataFrame()
        columns = ['Name', 'transit', 'Significance', 'RA', 'DEC']
        
        for i, Data in zip(columns, data):
            df[i] = Data
        
        # Agregar una columna para identificar el valor de psf
        df['PSF'] = psf
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
#!/usr/bin/env python3
# AUTHOR: Sara Fraija
# This script allows generating bash files to compute upper limits for each GRB. 
# You can change the GRB list, energy ranges, and EBL model.

# import pandas as pd
# import subprocess
# import os
# # ------------------------------------------------------------------------
# # PATHS & FILENAMES
# # ------------------------------------------------------------------------
# energy_ranges = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Energy_ranges.csv'
# GRBINFO_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/'
# OUTPUT_folder = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/files/2.5/PSF_0.25/'
# os.makedirs(OUTPUT_folder, exist_ok=True)
# PATH_SH = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/'
# RECALCULATED = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/corrected/'

# # Leer Energy_ranges.csv con nombres de columnas correctos
# Energy_ranges = pd.read_csv(energy_ranges, names=['Name', 'Emin', 'Emax'])
# Energy_ranges = Energy_ranges.set_index('Name')
# Energy_ranges.sort_values(by='Name')
# # ------------------------------------------------------------------------
# # Function to generate bash script
# # ------------------------------------------------------------------------
# def script(GRBINFO, transit, PATH_SH, OUTPUT_folder, RECALCULATED, EBL, PSF):
#     OUTPUT_FILE = f"{OUTPUT_folder}UpperLimit_{transit}_{EBL}.csv"

#     # Leer archivo CSV con GRB information
#     Data = pd.read_csv(GRBINFO)
#     Data.sort_values('Name')
#     # Verificar si la columna "Name" existe en el CSV
#     if "Name" not in Data.columns:
#         print(f"Error: La columna 'Name' no está en {GRBINFO}. Verifica el formato del archivo.")
#         print(f"Columnas disponibles: {Data.columns}")
#         return

#     # Usar "Name" como índice
#     Data = Data.set_index("Name")

#     # Asignar valores de Emin y Emax si existen en Energy_ranges
#     Data['Emin'] = Data.index.map(Energy_ranges['Emin'])
#     Data['Emax'] = Data.index.map(Energy_ranges['Emax'])
#     # Verificar si hay valores NaN en Emin o Emax
#     if Data[['Emin', 'Emax']].isna().any().any():
#         print("Error: Algunos GRBs no tienen valores de Emin o Emax. Verifica Energy_ranges.csv.")
#         print(Data[['Emin', 'Emax']].isna().sum())
#         return

#     line = ""
#     for grb in Data.index:  
#         row = Data.loc[grb]
#         RA = row['RA']
#         Dec = row['DEC']
#         z = row['z']
        
#         index = 2.07 if grb == 'GRB170817529' else 2.5

#         Emin = row['Emin']
#         Emax = row['Emax']
#         line += f'PL="PowerLaw,1.0e-10,{index}"\n'
#         line_1 = f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB0C1_N1024.fits.gz $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{1..10}}C{{0..1}}_N1024.fits.gz -b B0C1 B{{1..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
#         line_2 = f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{0..10}}C{{0..1}}_N1024.fits.gz -b B{{0..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
#         line += line_1 if grb == 'GRB150110923' else line_2
#         line += f'echo -e "{grb},${{ULs}}" >> $output_file\n'
        
#     # Código base del script Bash
#     code = f"""#! /usr/bin/env bash
# #SBATCH --mem-per-cpu=4GB
# #SBATCH -o {PATH_SH}/logs/UL.out
# #SBATCH --mail-type=all
# #SBATCH --mail-user=smfraijac@astro.unam.mx
# #SBATCH -J UL
# # Author: Sara Fraija
# source /data/disk01/home/smfraijac/.bashrc
# hawc_aerie
# output_file={OUTPUT_FILE}
# DR51=/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/config/pass5.f_DR/zebra-100pct-FHit.root
# dir={RECALCULATED}\n
# output_file={OUTPUT_FILE}\n
# rm -f output_file
# """
#     # Guardar el script Bash
#     file_path = f"{PATH_SH}UL{transit}_{EBL}_{PSF}.sh"
#     if os.path.exists(file_path):
#         os.remove(file_path)
#     with open(file_path, 'w') as bash_file:
#         bash_file.write(code + line)
#     print(f"Script generado: {file_path}")
# # ------------------------------------------------------------------------
# # Running script
# # ------------------------------------------------------------------------
# EBL_list = ['Franceschini08', 'Gilmore12Fiducial']

# # for EBL in EBL_list:
# #     File_1st_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates_with_Max_Sig_1_00.25.csv'
# #     File_2nd_transit = '/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/config/Coordinates_with_Max_Sig_2_00.25.csv'
# #     script(File_1st_transit, transit=1, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
# #     script(File_2nd_transit, transit=2, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF=0.25)
# #     p1 = subprocess.Popen(['bash', f"{PATH_SH}UL1_{EBL}_0.25.sh"])
# #     p2 = subprocess.Popen(['bash', f"{PATH_SH}UL2_{EBL}_0.25.sh"])


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line
Traceback (most recent call last):
  File "/data/disk01/home/smfraijac/hawc_software/hawc_aerie/aerie/install/bin/plotMercator.py", line 1114, in <module>
    main()
  File "/data/disk01/home/smfraijac/hawc_software/hawc_aerie/aerie/install/bin/plotMercator.py", line 642, in main
    return_projected_map=True)
  File "/data/disk01/home/smfraijac/hawc_software/miniconda3/envs/aerie_env/lib/python2.7/site-packages/healpy/visufunc.py", line 526, in cartview
    map = pixelfunc.ma_to_array(map)
  File "/data/disk01/home/smfraijac/hawc_software/miniconda3/envs/aerie_env/lib/python2.7/site-packages/healpy/pixelfunc.py", line 233, in ma_to_array
    return m.filled()
  File "/data/disk01/home/smfraijac/hawc_software/miniconda3/envs/aerie_env/lib/python2.7/site-packages/numpy/ma/core.py", line 3724, in filled
    np.copyto(result, fill_value, where=m)
KeyboardInterrupt


KeyboardInterrupt: 